# Spickzettel für KA

## Datein
> client.py
> server.py  
> model.py  
> Html_datei.html  

## Imports
> from flask import Flask, request, jsonify  
> from flask_cors import CORS  
> from pydantic import ValidationError  
> from model import Spieler  
> import json  
> import os  
--> in Server

> from pydantic import BaseModel, Field, field_validator, ValidationError  
--> In model  

> import json
> import requests
--> client

# model.py

Hier kommen alle moglichen atributte rein (name, alter) mit passender bezeichnung (str, int)  
class Spieler(BaseModel):   
-----------> allte atribute: die bezeichnung = Field(einschränkungen=,)    
----------->model_config = {"validate_assignment": True}  



| Option | Bedeutung | Beispiel | Wirkung |
|:--------|:-----------|:----------|:---------|
| `default` | Standardwert für das Feld | `Field(default=0)` | Wenn kein Wert übergeben wird, wird 0 gesetzt |
| `ge` | „greater or equal“ – Minimalwert | `Field(..., ge=0)` | Wert muss ≥ 0 sein |
| `gt` | „greater than“ – strenger Minimalwert | `Field(..., gt=0)` | Wert muss > 0 sein |
| `le` | „less or equal“ – Maximalwert | `Field(..., le=100)` | Wert muss ≤ 100 sein |
| `lt` | „less than“ – strenger Maximalwert | `Field(..., lt=100)` | Wert muss < 100 sein |
| `min_length` | Minimale Länge von Strings oder Listen | `Field(..., min_length=3)` | Prüft, dass Text oder Liste mindestens 3 Elemente hat |
| `max_length` | Maximale Länge von Strings oder Listen | `Field(..., max_length=20)` | Prüft, dass Text oder Liste höchstens 20 Elemente hat |

# client




URL = "http://localhost:12345/Spieler"

spieler_liste = []
##### Beispiel-Daten

spieler_daten = {
    "name": str/int (input("Name: ")),
}




response = requests.post(URL, json=spieler_daten)
print("Statuscode:", response.status_code)

if response.status_code == 201:👍
    print(json.dumps(response.json(), indent=4, ensure_ascii=False))
else:👎
    print(json.dumps(response.json(), indent=4, ensure_ascii=False))


# server

app = Flask(__name__)
CORS(app)

def attribute():
    return f"""
    Method: {request.method}
    Headers: {dict(request.headers)}
    Args: {request.args}
    Form: {request.form}
    Data: {request.data}
    Cookies: {request.cookies}
    Path: {request.path}
    URL: {request.url}
    Remote Address: {request.remote_addr}
    """

@app.route('/')
def home():
    response_attribut = attribute()
    return jsonify({"attribut": response_attribut})

@app.route('/Profil')
def gym():
    return "<html><body><h1>baum.</h1></body></html>"

@app.route('/message', methods=['POST'])
def handle_message():
    data = request.json or {}
    message = data.get('message', '')
    print(f"Empfangen: {message}")
    print(attribute())
    response_message = f"Echo: {message}"
    return jsonify({"response": response_message})




@app.route("/Spieler", methods=["POST"])
def handle_Spieler():
    """Erstellt einen neuen Spieler aus JSON-Daten und prüft es mit Pydantic."""
    try:
        data = request.get_json() or {}
        spieler = Spieler(**data)
        spieler_liste.append(spieler)
        with open("spieler.json", "w", encoding="utf-8") as f:
            json.dump([s.model_dump() for s in spieler_liste], f, ensure_ascii=False, indent=4)
        return jsonify({
            "status": "ok👍",
            "message": "Spieler erfolgreich erstellt🤝",
            "Spieler": spieler.model_dump()
        }), 201
    except ValidationError as e:
        return jsonify({
            "status": "error",
            "message": "Validierung fehlgeschlagen",
            "details": e.errors()
        }), 400
    except Exception as e:
        return jsonify({
            "status": "error",
            "message": "Interner Serverfehler",
            "details": str(e)
        }), 500
    

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000, debug=True)